# Building image cars driving data

## Importing Data

In [19]:
import torch as tch
import numpy as np
import plotly.graph_objects as go
import os
import cv2
from pathlib import Path

### Loading and building data

#### Determining the size

The images do not have the same shape so we will crop it to the minimum for all dimensions.

In [20]:
dir = Path("%pwd").resolve().parent
path_data = dir / "data_semantics/training/image_2/"
path_data

WindowsPath('C:/Users/blidi/OneDrive/Bureau/projet/semantic_segmentation/data_semantics/training/image_2')

In [21]:
# Vecteur pour stocker les tailles minimales des images
min_image_sizes = []

# Parcours de chaque fichier dans le répertoire
for filename in os.listdir(path_data):
    # Vérifiez si le fichier est une image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construisez le chemin complet du fichier
        filepath = os.path.join(path_data, filename)
        # Utilisez la déclaration with pour ouvrir le fichier
        with open(filepath, 'rb') as f:    
            # Chargez l'image avec OpenCV
            img = cv2.imread(filepath)
            # Convertissez l'image en BGR en RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Ajoutez la taille de l'image au vecteur des tailles minimales
            min_image_sizes.append(img.shape)

# Convertissez le vecteur des tailles minimales en un tenseur PyTorch
min_image_sizes = tch.tensor(min_image_sizes).min(0).values # don't need the indices
min_image_sizes

tensor([ 370, 1224,    3])

Same thing for the masks.

In [22]:
dir = Path("%pwd").resolve().parent
path_mask = dir / "data_semantics/training/semantic_rgb"
path_mask

WindowsPath('C:/Users/blidi/OneDrive/Bureau/projet/semantic_segmentation/data_semantics/training/semantic_rgb')

In [23]:
# Vecteur pour stocker les tailles minimales des images
min_mask_sizes = []

# Parcours de chaque fichier dans le répertoire
for filename in os.listdir(path_mask):
    # Vérifiez si le fichier est une image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construisez le chemin complet du fichier
        filepath = os.path.join(path_mask, filename)
        # Utilisez la déclaration with pour ouvrir le fichier
        with open(filepath, 'rb') as f:    
            # Chargez l'image avec OpenCV
            img = cv2.imread(filepath)
            # Convertissez l'image en BGR en RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Ajoutez la taille de l'image au vecteur des tailles minimales
            min_mask_sizes.append(img.shape)

# Convertissez le vecteur des tailles minimales en un tenseur PyTorch
min_mask_sizes = tch.tensor(min_mask_sizes).min(0).values # don't need the indices
min_mask_sizes

tensor([ 370, 1224,    3])

#### Cropping the images and building the data_tensor

In [24]:
print('The height is', int(min_image_sizes[0]), 'for the images and', int(min_mask_sizes[0]), 'for the masks.\n' 
      'The width is', int(min_image_sizes[1]), 'for the images and', int(min_mask_sizes[1]), 'for the masks.')
# Create a cell to verify that the two masks are equal, if not create a policy
h_size, w_size, _ = min_image_sizes
h_size = h_size - 2 # Check Creating the CNN part to see why

The height is 370 for the images and 370 for the masks.
The width is 1224 for the images and 1224 for the masks.


As it is driving images, we will crop it from the top for the height (because generally they are no so much information for cars in the sky) and from both side for the width. \\
For the width part, if the reducing value (w_img - w_size) is odd, we will minus the exceeding one from the left because cars drive on the right side so the citizens come most rapidly from the right (so we need the more data on that part).

In [25]:
def crop_img(img, h_size = h_size, w_size = w_size) :
    h, w, _ = img.shape
    reduce_h, reduce_w = h-h_size, w-w_size
    # it might be more efficient to add if instances to see if it necessary to crop the images or not
    return img[reduce_h:, int(np.ceil(reduce_w/2)):w-int(np.floor(reduce_w/2))]

In [26]:
# Liste pour stocker les images en tant que tenseurs
images_tensor = []

# Parcours de chaque fichier dans le répertoire
for filename in os.listdir(path_data):
    # Vérifiez si le fichier est une image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construisez le chemin complet du fichier
        filepath = os.path.join(path_data, filename)
        # Utilisez la déclaration with pour ouvrir le fichier
        with open(filepath, 'rb') as f:    
            # Chargez l'image avec OpenCV
            img = cv2.imread(filepath)
            # Convertissez l'image en BGR en RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Cropping the image
            img = crop_img(img)
            # Convertissez l'image en tenseur Pytorch
            img_tensor = tch.from_numpy(img.transpose(2, 0, 1))
            img_tensor = img_tensor.float() / 255.0  # Normalisez les valeurs des pixels entre 0 et 1
            # Ajoutez le tenseur de l'image à la liste
            images_tensor.append(img_tensor)

# Convertissez la liste de tenseurs en un seul tenseur contenant toutes les images
images_tensor = tch.stack(images_tensor)

In [27]:
# Liste pour stocker les images en tant que tenseurs
mask_tensor = []

# Parcours de chaque fichier dans le répertoire
for filename in os.listdir(path_mask):
    # Vérifiez si le fichier est une image
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Construisez le chemin complet du fichier
        filepath = os.path.join(path_mask, filename)
        # Utilisez la déclaration with pour ouvrir le fichier
        with open(filepath, 'rb') as f:    
            # Chargez l'image avec OpenCV
            img = cv2.imread(filepath)
            # Convertissez l'image en BGR en RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Cropping the image
            img = crop_img(img)
            # Convertissez l'image en tenseur Pytorch
            img_tensor = tch.from_numpy(img.transpose(2, 0, 1))
            img_tensor = img_tensor.float() / 255.0  # Normalisez les valeurs des pixels entre 0 et 1
            # Ajoutez le tenseur de l'image à la liste
            mask_tensor.append(img_tensor)

# Convertissez la liste de tenseurs en un seul tenseur contenant toutes les images
mask_tensor = tch.stack(mask_tensor)

In [28]:
training_tensor = tch.stack((images_tensor, mask_tensor)) 
training_tensor.shape

torch.Size([2, 200, 3, 368, 1224])

Viewing a couple of images to see if everything is ok (it may be better to view it with plt...).

In [29]:
# HEAVY CELL
# ind=np.random.randint(0, 199, 4)
# fig = go.Figure()
# c=0
# for i in ind : 
#     # Transpose back the images to the good dimensions
#     fig.add_trace(go.Image(z=np.concatenate((training_tensor[0,i].permute(1, 2, 0)*255, training_tensor[1,i].permute(1, 2, 0)*255),1), y0 = c*h_size))
#     c+=1
# fig.show()

### Exporting the tensor

In [30]:
# Datasets saving

# tch.save(training_tensor, Path("%pwd").resolve().parent / "data" / "training_tensor.pt")